In [39]:
import os
from dotenv import load_dotenv

# Load biến môi trường từ file .env (file .env cần nằm trong cùng thư mục với notebook)
load_dotenv()

# Lấy giá trị của TOGETHER_API_KEY từ biến môi trường
together_api_key = os.getenv('TOGETHER_API_KEY')
if not together_api_key:
    raise ValueError("Chưa tìm thấy TOGETHER_API_KEY trong file .env!")

elastic_api_key = os.getenv('ELASTIC_API_KEY')
if not elastic_api_key:
    raise ValueError("Chưa tìm thấy ELASTIC_API_KEY trong file .env!")

elastic_cloud_id = os.getenv('ELASTIC_CLOUD_ID')
if not elastic_cloud_id:
    raise ValueError("Chưa tìm thấy ELASTIC_CLOUD_ID trong file .env!")

tavily_api_key = os.getenv('TAVILY_API_KEY')
if not tavily_api_key:
    raise ValueError("Chưa tìm thấy TAVILY_API_KEY trong file .env!")


serper_api_key = os.getenv('SERPER_API_KEY')
if not serper_api_key:
    raise ValueError("Chưa tìm thấy SERPER_API_KEY trong file .env!")


print(" Key đã được thiết lập thành công!")


 Key đã được thiết lập thành công!


In [40]:
print(together_api_key)
print(elastic_api_key)
print(elastic_cloud_id)

ba4066853f6d39efbdb8e58274802acdb950955adb52d523b36cad88c5493aa1
LUVTcUlKWUIyR2JxbU84a1dRYVo6TFhidjQ4VlVRRmFYNUJQY2MweFVEUQ==
My_Elasticsearch_deployment:YXAtZWFzdC0xLmF3cy5lbGFzdGljLWNsb3VkLmNvbTo0NDMkNjAyNDBhMzU5NGU5NDIzNjkzMmJjYjQwMWYyZWZmMTMkNmQxNjEyNjAwZWI1NDQzZmE2ODA1YTI1YWQ5M2Y3ZTA=


In [41]:
from langchain_together import ChatTogether, TogetherEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
llm = ChatTogether(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
)
#" )
result = llm.invoke("Thủ đô Việt Nam là thành phố nào? ")
print(result.content)

Thủ đô của Việt Nam là Hà Nội.


In [42]:
# from tqdm.notebook import tqdm
embd = HuggingFaceEmbeddings(model_name="VoVanPhuc/sup-SimCSE-VietNamese-phobert-base")

No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.


In [43]:
embd

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='VoVanPhuc/sup-SimCSE-VietNamese-phobert-base', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Load Elasticsearch Store



In [44]:
from langchain.vectorstores import ElasticsearchStore
index_name = "ecom-law0"

In [45]:
# Khởi tạo connection tới Elasticsearch thông qua Elastic Cloud
vector_store = ElasticsearchStore(
    es_cloud_id=elastic_cloud_id,
    es_api_key=elastic_api_key,
    index_name=index_name,
    embedding=embd,
)

In [46]:
indices = vector_store.client.cat.indices(format="json")
for index_info in indices:
    print(index_info["index"])


.internal.alerts-transform.health.alerts-default-000001
.internal.alerts-observability.logs.alerts-default-000001
.internal.alerts-observability.uptime.alerts-default-000001
.ds-metrics-fleet_server.agent_versions-default-2025.04.10-000001
.internal.alerts-ml.anomaly-detection.alerts-default-000001
.internal.alerts-observability.slo.alerts-default-000001
ecom-law0
.internal.alerts-default.alerts-default-000001
new-index
.internal.alerts-observability.apm.alerts-default-000001
.elastic-connectors-v1
.internal.alerts-observability.metrics.alerts-default-000001
.internal.alerts-ml.anomaly-detection-health.alerts-default-000001
.internal.alerts-observability.threshold.alerts-default-000001
.ds-metrics-fleet_server.agent_status-default-2025.04.10-000001
.elastic-connectors-sync-jobs-v1
.internal.alerts-security.alerts-default-000001
.ds-logs-enterprise_search.audit-default-2025.04.10-000001
.ds-logs-enterprise_search.api-default-2025.04.10-000001
.internal.alerts-stack.alerts-default-000001

In [47]:
num_docs = vector_store.client.count(index=index_name)['count']
print(f"Number of documents in the vectorstore: {num_docs}")


Number of documents in the vectorstore: 318


## Hybrid Retrieval

In [48]:
from typing import Dict
from langchain_elasticsearch import ElasticsearchRetriever
from elasticsearch import Elasticsearch

In [49]:
index_name = "ecom-law0"
text_field = "text"
es_url = "https://my-elasticsearch-deployment.es.ap-east-1.aws.elastic-cloud.com"

In [50]:
# --- Định nghĩa hàm hybrid_query ---
def hybrid_query(search_query: str) -> Dict:
    # Sử dụng embed_query để tạo vector cho truy vấn
    query_vector = embd.embed_query(search_query)
    return {
        "retriever": {
            "rrf": {  # Rank Reciprocal Fusion: kết hợp hai truy vấn theo kiểu "should"
                "retrievers": [
                    {  # Thành phần full text search: dùng match query trên trường "text"
                        "standard": {
                            "query": {
                                "match": {
                                    text_field: search_query
                                }
                            }
                        }
                    },
                    {  # Thành phần vector search: dùng knn query trên trường "embedding_field"
                        "knn": {
                            "field": "embedding_field",  # Điều chỉnh nếu mapping của bạn dùng tên khác
                            "query_vector": query_vector,
                            "k": 5,
                            "num_candidates": 10
                        }
                    }
                ]
            }
        }
    }


In [51]:
# --- Thiết lập Elasticsearch client với xác thực ---
from elasticsearch import Elasticsearch
headers = {"Authorization": "ApiKey " + elastic_api_key.strip()}
es_client = Elasticsearch(
    hosts=[es_url],
    headers=headers
)
print("ElasticSearch Cluster Info:")
print(es_client.info())

ElasticSearch Cluster Info:
{'name': 'instance-0000000001', 'cluster_name': '60240a3594e94236932bcb401f2eff13', 'cluster_uuid': 'nvcaKx7WRZCEdKsZ-3nr0w', 'version': {'number': '8.17.4', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1', 'build_date': '2025-03-20T15:39:59.811110136Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [52]:
from langchain_elasticsearch import ElasticsearchRetriever
hybrid_retriever = ElasticsearchRetriever.from_es_params(
    index_name=index_name,
    body_func=hybrid_query,
    content_field=text_field,  # Trường chứa nội dung văn bản trong index
    url=es_url,
    api_key=elastic_api_key.strip()
)


## Prompting

In [53]:
from langchain.prompts import PromptTemplate

prompt_template = """
Bạn là một chuyên gia pháp lý hàng đầu, chuyên sâu về **Luật Thương mại của Việt Nam**, đồng thời am hiểu đầy đủ và chính xác các **bộ luật khác của Việt Nam** như: Bộ luật Dân sự, Luật Doanh nghiệp, Luật Trọng tài thương mại, Luật Đầu tư, Luật Sở hữu trí tuệ, v.v.

Bạn có nhiều năm kinh nghiệm giảng dạy, tư vấn và giải quyết tranh chấp pháp lý trong môi trường thực tiễn tại Việt Nam.

---

### ❓ Câu hỏi pháp lý:
{query}

---

### 📚 Thông tin pháp lý được truy xuất từ văn bản:
(Chỉ sử dụng thông tin trong phần này. Nếu không đủ căn cứ, hãy nói rõ.)
------------------------------------------------
{retrieved_context}
------------------------------------------------

---

### 🎯 Nhiệm vụ của bạn:

1. **Phân tích câu hỏi** để xác định vấn đề pháp lý cốt lõi và các quy định cần xem xét.
2. **Chỉ dựa trên thông tin được truy xuất ở trên**, đưa ra phân tích pháp lý chi tiết, có **trích dẫn đúng điều/khoản/bộ luật** theo định dạng chuẩn: `"Điều X, Khoản Y, Luật Z: ..."`.
3. **Tuyệt đối không suy diễn hoặc tự tạo nội dung luật** nếu không có trong context.
4. Nếu có nhiều cách hiểu/phương án, hãy liệt kê và **so sánh khách quan** (ưu/nhược điểm).
5. **Đưa ví dụ minh họa** cụ thể nếu có thể.
6. **Tổng hợp và trả lời rõ ràng, có logic**, trình bày theo **cấu trúc với tiêu đề phụ**.

---

### 📝 Trả lời:
"""


In [54]:
prompt = PromptTemplate(template=prompt_template, input_variables=["retrieved_context", "query"])

## Post Processing

In [55]:
import re
def postprocess_legal_output(raw_output: str) -> str:
    # Tách các phần bằng tiêu đề hoặc keyword thường gặp
    question_section = ""
    law_section = ""
    analysis_section = ""
    conclusion_section = ""
    example_section = ""

    # 1. Trích câu hỏi
    q_match = re.search(r"(Câu hỏi pháp lý.*?:)(.*?)(\n|$)", raw_output, re.IGNORECASE)
    if q_match:
        question_section = q_match.group(2).strip()
    else:
        question_section = "Không xác định được câu hỏi."

    # 2. Trích điều luật
    law_match = re.search(r"(Điều\s+\d+.*?:.*?)\n", raw_output, re.IGNORECASE | re.DOTALL)
    if law_match:
        law_section = law_match.group(1).strip()

    # 3. Trích ví dụ minh họa
    example_match = re.search(r"(Ví dụ.*?:)(.*?)(Kết luận|$)", raw_output, re.IGNORECASE | re.DOTALL)
    if example_match:
        example_section = example_match.group(2).strip()

    # 4. Trích kết luận
    conclusion_match = re.search(r"(Kết luận.*?:?)(.*)", raw_output, re.IGNORECASE | re.DOTALL)
    if conclusion_match:
        conclusion_section = conclusion_match.group(2).strip()

    # 5. Phần phân tích còn lại (giữa luật và kết luận)
    body_match = re.search(r"(?:Điều.*?\n)(.*?)(Ví dụ|Kết luận|$)", raw_output, re.IGNORECASE | re.DOTALL)
    if body_match:
        analysis_section = body_match.group(1).strip()

    # 6. Build lại format markdown rõ ràng
    result = f"""
## ❓ Phân tích câu hỏi
Câu hỏi pháp lý đặt ra là:  
**{question_section}**

---

## ⚖️ Phân tích pháp lý
> **{law_section}**

📌 {analysis_section}

---

## 🧠 Ví dụ minh họa
> {example_section}

---

## ✅ Kết luận
> {conclusion_section}
""".strip()

    return result

## Retrieval


### Test 1

In [56]:
# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?"
retrieved_docs = hybrid_retriever.invoke(query)

In [57]:
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print(retrieved_context)

18 CÔNG BÁO Số 1251 + 1252 Ngày 29-12-2015 Tòa án đã ra quyết định tuyên bố một người mất tích hoặc là đã chết có thẩm quyền giải quyết yêu cầu hủy bỏ quyết định tuyên bố một người mất tích hoặc là đã chết; d) Tòa án nơi người phải thi hành bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoài cư trú, làm việc, nếu người phải thi hành án là cá nhân hoặc nơi người phải thi hành án có trụ sở, nếu người phải thi hành án là cơ quan, tổ chức hoặc nơi có tài sản liên quan đến việc thi hành bản án, quyết định của Tòa án nước ngoài có thẩm quyền giải quyết yêu cầu công nhận và cho thi hành tại Việt Nam hoặc không công nhận bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoài; đ) Tòa án nơi người gửi đơn cư trú, làm việc, nếu người gửi đơn là cá nhân hoặc nơi người gửi đơn có trụ sở, nếu người gửi đơn là cơ quan, tổ chức có thẩm quyền giải quyết yêu cầu không công nhận bản án, quyết định dân sự,

In [58]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)


In [59]:
print(final_prompt)


Bạn là một chuyên gia pháp lý hàng đầu, chuyên sâu về **Luật Thương mại của Việt Nam**, đồng thời am hiểu đầy đủ và chính xác các **bộ luật khác của Việt Nam** như: Bộ luật Dân sự, Luật Doanh nghiệp, Luật Trọng tài thương mại, Luật Đầu tư, Luật Sở hữu trí tuệ, v.v.

Bạn có nhiều năm kinh nghiệm giảng dạy, tư vấn và giải quyết tranh chấp pháp lý trong môi trường thực tiễn tại Việt Nam.

---

### ❓ Câu hỏi pháp lý:
Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?

---

### 📚 Thông tin pháp lý được truy xuất từ văn bản:
(Chỉ sử dụng thông tin trong phần này. Nếu không đủ căn cứ, hãy nói rõ.)
------------------------------------------------
18 CÔNG BÁO Số 1251 + 1252 Ngày 29-12-2015 Tòa án đã ra quyết định tuyên bố một người mất tích hoặc là đã chết có thẩm quyền giải quyết yêu cầu hủy bỏ quyết định tuyên bố một người mất tích hoặc là đã chết; d) Tòa án nơi người phải thi hành bản án, quyết định dân sự, hôn nhân và gia đình, kinh doanh, thương mại, lao động của Tòa án nước ngoà

In [60]:
response = llm(final_prompt)
print(response.content)

**Trọng tài nước ngoài là gì?**

Theo quy định tại Điều 3, Khoản 11, Luật Trọng tài thương mại: "Trọng tài nước ngoài là Trọng tài được thành lập theo quy định của pháp luật trọng tài nước ngoài do các bên thỏa thuận lựa chọn để tiến hành giải quyết tranh chấp ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam."

Như vậy, trọng tài nước ngoài là một tổ chức trọng tài được thành lập và hoạt động theo pháp luật của nước ngoài, nhưng có thể tiến hành giải quyết tranh chấp tại Việt Nam nếu các bên có thỏa thuận lựa chọn.

**Ví dụ minh họa:**

Công ty A của Việt Nam và Công ty B của Nhật Bản có một hợp đồng thương mại và thỏa thuận giải quyết tranh chấp bằng trọng tài tại Tokyo, Nhật Bản. Trong trường hợp này, trọng tài Tokyo sẽ được coi là trọng tài nước ngoài vì nó được thành lập và hoạt động theo pháp luật của Nhật Bản, nhưng vẫn có thể giải quyết tranh chấp giữa hai công ty tại Việt Nam.


### Test 2

In [63]:

# --- Thực hiện retrieval ---
query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [64]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [65]:
response = llm(final_prompt)
print(response.content)

**Quyết định trọng tài**

Theo Luật Trọng tài thương mại, quyết định trọng tài là kết quả cuối cùng của quá trình giải quyết tranh chấp bằng trọng tài. 

**Thẩm quyền giải quyết tranh chấp của Trọng tài**

Điều 2, Luật Trọng tài thương mại quy định về thẩm quyền giải quyết tranh chấp của Trọng tài như sau:

"1. Tranh chấp giữa các bên phát sinh từ hoạt động thương mại.
2. Tranh chấp phát sinh giữa các bên trong đó ít nhất một bên có hoạt động thương mại.
3. Tranh chấp khác giữa các bên mà pháp luật quy định được giải quyết bằng Trọng tài."

**Quyết định trọng tài**

Không có quy định cụ thể về quyết định trọng tài trong các văn bản được cung cấp. Tuy nhiên, có thể hiểu rằng quyết định trọng tài là kết quả cuối cùng của quá trình giải quyết tranh chấp bằng trọng tài, được đưa ra bởi hội đồng trọng tài sau khi xem xét các bằng chứng và lập luận của các bên.

**Giải quyết tranh chấp trong thương mại điện tử**

Điều 76, Nghị định số 52/2013/NĐ-CP quy định về giải quyết tranh chấp trong thư

### Test 3

In [66]:
# --- Thực hiện retrieval ---
query = '''Theo luật trọng tài thương mại,Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài có được áp dụng 
pháp luật Việt Nam để giải quyết tranh chấp hay không ?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [67]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)

In [68]:
response = llm(final_prompt)
print(response.content)

**Phân tích câu hỏi**

Câu hỏi pháp lý đặt ra là: Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài có được áp dụng pháp luật Việt Nam để giải quyết tranh chấp hay không?

**Phân tích pháp lý**

Theo quy định tại Điều 14, Luật Trọng tài thương mại, đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài áp dụng pháp luật Việt Nam để giải quyết tranh chấp.

"Cụ thể, Điều 14, Luật Trọng tài thương mại quy định:

1. Đối với tranh chấp không có yếu tố nước ngoài, Hội đồng trọng tài áp dụng pháp luật Việt Nam để giải quyết tranh chấp.
2. Đối với tranh chấp có yếu tố nước ngoài, Hội đồng trọng tài áp dụng pháp luật do các bên lựa chọn; nếu các bên không có thỏa thuận về luật áp dụng thì Hội đồng trọng tài quyết định áp dụng pháp luật mà Hội đồng trọng tài cho là phù hợp nhất.
3. Trường hợp pháp luật Việt Nam, pháp luật do các bên lựa chọn không có quy định cụ thể liên quan đến nội dung tranh chấp thì Hội đồng trọng tài được áp dụng tập quán quốc tế để giải quyết tra

### Test 4

In [69]:
# --- Thực hiện retrieval ---
query = '''Luật trọng tài thương mại do ai ban hành'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [70]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Luật Trọng tài thương mại do ai ban hành**

Không có thông tin cụ thể về việc Luật Trọng tài thương mại do ai ban hành trong văn bản được cung cấp. Tuy nhiên, thông thường, các bộ luật và nghị định ở Việt Nam được ban hành bởi Quốc hội hoặc Chính phủ.

**Tìm kiếm thông tin liên quan**

Trong văn bản được cung cấp, có một số thông tin liên quan đến thương mại điện tử, bảo vệ quyền lợi người tiêu dùng, và các quy định về hoạt động thương mại điện tử. Tuy nhiên, không có thông tin cụ thể về Luật Trọng tài thương mại.

**Kết luận**

Do không có thông tin cụ thể về việc Luật Trọng tài thương mại do ai ban hành trong văn bản được cung cấp, chúng tôi không thể đưa ra câu trả lời chính xác. Tuy nhiên, thông thường, các bộ luật và nghị định ở Việt Nam được ban hành bởi Quốc hội hoặc Chính phủ.


### Test 5

In [71]:
# --- Thực hiện retrieval ---
query = '''Phán quyết của trọng tài trong luật trọng tài thương mại có hiệu lực như thế nào'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [72]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Phán quyết của trọng tài trong luật trọng tài thương mại có hiệu lực như thế nào**

**1. Phạm vi điều chỉnh của Luật Trọng tài thương mại**

Theo Điều 1, Luật Trọng tài thương mại quy định về thẩm quyền của Trọng tài thương mại, các hình thức trọng tài, tổ chức trọng tài, Trọng tài viên; trình tự, thủ tục trọng tài; quyền, nghĩa vụ và trách nhiệm của các bên trong tố tụng trọng tài; thẩm quyền của Tòa án đối với hoạt động trọng tài; tổ chức và hoạt động của Trọng tài nước ngoài tại Việt Nam, thi hành phán quyết trọng tài.

**2. Thẩm quyền giải quyết các tranh chấp của Trọng tài**

Theo Điều 2, Luật Trọng tài thương mại quy định rằng Trọng tài có thẩm quyền giải quyết các tranh chấp sau:

* Tranh chấp giữa các bên phát sinh từ hoạt động thương mại.
* Tranh chấp phát sinh giữa các bên trong đó ít nhất một bên có hoạt động thương mại.
* Tranh chấp khác giữa các bên mà pháp luật quy định được giải quyết bằng Trọng tài.

**3. Hiệu lực của phán quyết trọng tài**

Không có quy định cụ thể v

### Test 6

In [73]:
# --- Thực hiện retrieval ---
query = '''Sự khác biệt giữa trọng tài thương mại và toà án là gì'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [74]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Sự khác biệt giữa trọng tài thương mại và tòa án**

Trọng tài thương mại và tòa án là hai phương thức giải quyết tranh chấp khác nhau trong lĩnh vực kinh doanh, thương mại. Dưới đây là một số điểm khác biệt chính giữa hai phương thức này:

**1. Thẩm quyền giải quyết**

* Trọng tài thương mại: Theo Điều 2, Luật Trọng tài thương mại, trọng tài thương mại có thẩm quyền giải quyết các tranh chấp phát sinh từ hoạt động thương mại, bao gồm tranh chấp giữa các bên phát sinh từ hoạt động thương mại, tranh chấp phát sinh giữa các bên trong đó ít nhất một bên có hoạt động thương mại, và tranh chấp khác giữa các bên mà pháp luật quy định được giải quyết bằng trọng tài.
* Tòa án: Theo Điều 30, Bộ luật Dân sự, tòa án có thẩm quyền giải quyết các tranh chấp về kinh doanh, thương mại, bao gồm tranh chấp phát sinh trong hoạt động kinh doanh, thương mại giữa cá nhân, tổ chức có đăng ký kinh doanh với nhau và đều có mục đích lợi nhuận.

**2. Thủ tục giải quyết**

* Trọng tài thương mại: Thủ tục giải q

### Test 7

In [75]:
# --- Thực hiện retrieval ---
query = '''Trong luật trọng tài thương mại, Phán quyết trọng tài có giá trị pháp lí như thế nào?'''
retrieved_docs = hybrid_retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [76]:
final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
response = llm(final_prompt)
print(response.content)

**Giá trị pháp lý của Phán quyết trọng tài**

Phán quyết trọng tài có giá trị pháp lý như thế nào là một vấn đề quan trọng trong luật trọng tài thương mại. Theo quy định của Luật Trọng tài thương mại, phán quyết trọng tài có giá trị pháp lý như bản án của Tòa án.

**Điều 61, Luật Trọng tài thương mại:**

"Phán quyết trọng tài có hiệu lực pháp luật và được thi hành theo quy định của pháp luật về thi hành án dân sự."

**Điều 62, Luật Trọng tài thương mại:**

"Phán quyết trọng tài được công nhận và cho thi hành tại Việt Nam nếu đáp ứng các điều kiện sau:

a) Phán quyết trọng tài được ban hành bởi Trọng tài nước ngoài có thẩm quyền;

b) Phán quyết trọng tài không vi phạm các quy định của pháp luật Việt Nam về trọng tài thương mại;

c) Phán quyết trọng tài không trái với các nguyên tắc cơ bản của pháp luật Việt Nam."

Như vậy, phán quyết trọng tài có giá trị pháp lý như bản án của Tòa án và được thi hành theo quy định của pháp luật về thi hành án dân sự. Tuy nhiên, để được công nhận và cho 

### Test 8


In [77]:
# # --- Thực hiện retrieval ---
# query = "Theo luật trọng tài thương mại, Quyết định trọng tài?"
# retrieved_docs = hybrid_retriever.invoke(query)
# retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])

In [78]:
# final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
# response = llm(final_prompt)
# print(response.content)

In [79]:
# print(final_prompt)

## Search Tools

### Tavily Search

In [107]:
from langchain_community.tools import TavilySearchResults
from pydantic import BaseModel, Field
from typing import List

In [108]:
# Định nghĩa model cho một mục kết quả
class SearchResultItem(BaseModel):
    url: str = Field(..., description="Link của bài viết")
    content: str = Field(..., description="Nội dung tóm tắt hoặc câu trả lời")

# Định nghĩa model cho output của tool
class TavilySearchOutput(BaseModel):
    results: List[SearchResultItem] = Field(..., description="Danh sách kết quả tìm kiếm")

In [109]:
tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False,
    name="TavilyVietnamLawSearch",
    description=(
        "Công cụ tìm kiếm chuyên dụng để hỗ trợ tư vấn, phân tích và tra cứu pháp luật Việt Nam, "
        "đặc biệt tập trung vào Luật Thương mại, Luật Trọng tài thương mại, và các luật liên quan. "
        "Công cụ này cung cấp các đoạn nội dung từ văn bản pháp luật, hướng dẫn áp dụng luật, bài báo phân tích, "
        "và thông tin cập nhật từ các nguồn chính thống như thuvienphapluat.vn, vietnamlaw.net, báo pháp luật v.v."
    ),
)

In [110]:
def format_tavily_result(raw_result: list) -> str:
    formatted = "📚 **Kết quả từ Tavily:**\n\n"
    for idx, item in enumerate(raw_result, start=1):
        url = item.get("url", "Không có URL")
        content = item.get("content", "Không có nội dung").strip()
        source_type = (
            "official"
            if any(domain in url for domain in ["phapluat", "hethongphapluat", "lawnet", "moj.gov", "quochoi.vn"])
            else "news"
        )
        # Làm sạch content – ngắt dòng dài
        content = content.replace(". ", ".\n").replace("? ", "?\n").replace("! ", "!\n")
        
        # Format mỗi mục đẹp
        formatted += f"### {idx}. [{url}]({url})\n"
        formatted += f"🔗 **Loại nguồn:** `{source_type}`\n"
        formatted += f"**Tóm tắt nội dung:**\n{content}\n"
        formatted += "-" * 40 + "\n"
    return formatted

In [111]:
# Gọi tool với truy vấn tiếng Việt
raw_result = tool.invoke({
    "query": "theo luật trọng tài thương mại,trọng tài nước ngoài là gì?"
})

In [112]:
# In kết quả đã format
print(format_tavily_result(raw_result))

📚 **Kết quả từ Tavily:**

### 1. [https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/trong-tai-nuoc-ngoai-la-gi-van-phong-dai-dien-cua-to-chuc-trong-tai-nuoc-ngoai-tai-viet-nam-cham-du-82001.html](https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/trong-tai-nuoc-ngoai-la-gi-van-phong-dai-dien-cua-to-chuc-trong-tai-nuoc-ngoai-tai-viet-nam-cham-du-82001.html)
🔗 **Loại nguồn:** `official`
**Tóm tắt nội dung:**
Trọng tài nước ngoài là gì?

Căn cứ tại khoản 11 Điều 3 Luật Trọng tài thương mại 2010 định nghĩa trọng tài nước ngoài như sau:

Trọng tài nước ngoài là Trọng tài được thành lập theo quy định của pháp luật trọng tài nước ngoài do các bên thỏa thuận lựa chọn để tiến hành giải quyết tranh chấp ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam.
----------------------------------------
### 2. [https://thuvienphapluat.vn/phap-luat/ho-tro-phap-luat/trong-tai-nuoc-ngoai-la-gi-van-phong-dai-dien-cua-to-chuc-trong-tai-nuoc-ngoai-tai-viet-nam-cham-du-82001.html](https://thuvienphap

In [103]:
# raw_result

In [104]:
# def normalize_tavily_result_for_fact_check(raw_result: list) -> list:
#     normalized = []
#     for item in raw_result:
#         normalized.append({
#             "source": "Tavily",
#             "title": item.get("title") or item.get("url", "")[:50],
#             "url": item.get("url", ""),
#             "snippet": item.get("content", "")[:1000].replace("\n", " "),
#             "source_type": "official" if "phapluat" in item.get("url", "") else "news",
#             "date_published": None
#         })
#     return normalized

In [105]:
# structured_output = normalize_tavily_result_for_fact_check(raw_result)
# print(structured_output)

In [106]:

# structured_output

### Google Search

In [84]:
from langchain_community.utilities import GoogleSerperAPIWrapper
import pprint

In [85]:
search = GoogleSerperAPIWrapper(type="news")
results = search.results("Điều 3 của Luật định nghĩa “giao dịch điện tử” và “phương tiện điện tử” như thế nào?")
def format_news_results(results):
    if not results or "news" not in results:
        return "Không tìm thấy kết quả tin tức nào."

    formatted = "Kết quả Tìm kiếm Tin tức:\n\n"
    for idx, item in enumerate(results["news"], start=1):
        title = item.get("title", "Không có tiêu đề")
        snippet = item.get("snippet", "Không có mô tả")
        link = item.get("link", "Không có link")
        date = item.get("date", "Không có ngày")
        source = item.get("source", "Không có nguồn")
        formatted += f"{idx}. {title}\n"
        formatted += f"   - Nguồn: {source} | Ngày: {date}\n"
        formatted += f"   - Mô tả: {snippet}\n"
        formatted += f"   - Link: {link}\n\n"
    return formatted

# In kết quả đã được định dạng
print(format_news_results(results))

Kết quả Tìm kiếm Tin tức:

1. Bảo vệ quyền lợi người tiêu dùng trong hoạt động thương mại điện tử theo Luật Bảo vệ quyền lợi người tiêu dùng năm 2023
   - Nguồn: Tổng cục Quản lý thị trường | Ngày: 7 months ago
   - Mô tả: Cuộc cách mạng công nghiệp 4.0 đã khiến kinh doanh trực tuyến qua các nền tảng điện tử trở thành tất yếu. Giao dịch thương mại điện tử phát...
   - Link: https://dms.gov.vn/tin-chi-tiet/-/chi-tiet/bao-ve-quyen-loi-nguoi-tieu-dung-trong-hoat-%C4%91ong-thuong-mai-%C4%91ien-tu-theo-luat-bao-ve-quyen-loi-nguoi-tieu-dung-nam-2023-87436-1.html

2. Tội 'Sử dụng mạng viễn thông, mạng máy tính, phương tiện điện tử để chiếm đoạt tài sản' theo quy định của Bộ luật Hình sự 2015
   - Nguồn: Tạp chí Điện tử Luật sư Việt Nam | Ngày: 3 months ago
   - Mô tả: (LSVN) - Bài viết thể hiện quan điểm của các tác giả về dấu hiệu định tội của tội phạm sử dụng mạng viễn thông, mạng máy tính, phương tiện...
   - Link: https://lsvn.vn/toi-su-dung-mang-vien-thong-mang-may-tinh-phuong-tien-dien-

## Full Pipeline

In [117]:
def run_full_pipeline(query: str) -> str:
    # 1. Lấy kết quả từ RAG pipeline
    retrieved_docs = hybrid_retriever.invoke(query)
    retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    final_prompt = prompt.format(retrieved_context=retrieved_context, query=query)
    response = llm(final_prompt)  # Giả sử bạn đã khởi tạo đối tượng LLM (llm)
    rag_answer = response.content

    # 2. Lấy kết quả từ Tavily Search
    raw_tavily = tool.invoke({"query": query})
    tavily_output = format_tavily_result(raw_tavily)

    # 3. Lấy kết quả từ Google Serper Search
    serper_results = search.results(query)
    news_output = format_news_results(serper_results)

    # 4. Tổng hợp output
    full_output = f"""
==============================
## PHẦN I: Phân tích từ văn bản pháp luật (RAG pipeline)

{rag_answer.strip()}

==============================
## PHẦN II: Các nguồn thông tin bổ sung từ công cụ tìm kiếm

--- Tavily ---
{tavily_output.strip()}

--- Google Serper (Tin tức) ---
{news_output.strip()}

==============================
""".strip()
    return full_output



In [118]:
# Ví dụ sử dụng
user_query = "Theo luật trọng tài thương mại, trọng tài nước ngoài là gì?"
final_result = run_full_pipeline(user_query)

# In kết quả cuối cùng
print(final_result)

## PHẦN I: Phân tích từ văn bản pháp luật (RAG pipeline)

**Trọng tài nước ngoài là gì?**

Theo Điều 3, Khoản 11, Luật Trọng tài thương mại: "Trọng tài nước ngoài là Trọng tài được thành lập theo quy định của pháp luật trọng tài nước ngoài do các bên thỏa thuận lựa chọn để tiến hành giải quyết tranh chấp ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam."

**Giải thích:**

Trọng tài nước ngoài là một tổ chức trọng tài được thành lập và hoạt động theo pháp luật của một quốc gia khác ngoài Việt Nam. Các bên trong một tranh chấp có thể thỏa thuận lựa chọn trọng tài nước ngoài để giải quyết tranh chấp của mình, và trọng tài này có thể tiến hành giải quyết tranh chấp ở ngoài lãnh thổ Việt Nam hoặc trong lãnh thổ Việt Nam.

**Ví dụ minh họa:**

Một công ty Việt Nam và một công ty Nhật Bản có một tranh chấp về hợp đồng thương mại. Hai bên thỏa thuận lựa chọn trọng tài Nhật Bản để giải quyết tranh chấp. Trong trường hợp này, trọng tài Nhật Bản là một trọng tài nước ngoài vì nó được thành 